In [31]:
from src import object_render
from pathlib import Path
from src import model_save_load
import numpy as np
import open3d as o3d
data_dir = Path("./data/hammer_1")
depth_dir = data_dir/"depths"
render_dir = data_dir/"renders"
mesh_file = data_dir / "object_convex_decomposition.obj"
mesh_o3d, mesh_trimesh, info = model_save_load.load_mesh(mesh_file, vis=False)

In [32]:
mesh_o3d.translate(-mesh_o3d.get_center())


TriangleMesh with 623 points and 1230 triangles.

In [33]:
file_name = depth_dir/"depth_1.png"
depth_mask_file_name = depth_dir/"depth_mask_1.npy"
depth = o3d.io.read_image(file_name)

In [34]:
depth_mask = np.load(depth_mask_file_name)
mask = depth_mask.astype("bool")
depth_array = np.asarray(depth)
masked_depth_array = depth_array * depth_mask

In [37]:
depth_mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
masked_depth_array.shape

(1080, 1920)

In [36]:
o3d.io.write_image(data_dir/"masked_depth.png",o3d.geometry.Image(masked_depth_array))

RuntimeError: Image can only be initialized from buffer of uint8, uint16, or float!

In [38]:
from src import point_cloud_utils
depth_image_masked = o3d.io.read_image(data_dir/"masked_depth.png")
params = o3d.io.read_pinhole_camera_parameters(data_dir/"renders/camera_params.json")
labeling_pcd = o3d.geometry.PointCloud.create_from_depth_image(depth_image_masked, depth_scale=10, intrinsic=params.intrinsic, extrinsic=params.extrinsic, depth_trunc=1000.0)
labeling_pcd.paint_uniform_color([1, 0, 0])
directions = np.load(render_dir / "directions.npy")
max_ind_list = np.load(depth_dir / "max_ind_list.npy")
max_ind = max_ind_list[2]
direction, rot = directions[max_ind][:-1], directions[max_ind][-1]
point_cloud_utils.rotate_back(direction, rot, labeling_pcd)
o3d.visualization.draw_geometries([mesh_o3d, labeling_pcd])